<a href="https://colab.research.google.com/github/JoseVillagranE/IBGAN/blob/master/Copia_de_IBGAN_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn import Parameter, init
from torch.nn.utils import spectral_norm
from torch.distributions.multivariate_normal import MultivariateNormal
from itertools import chain

class UnFlatten(nn.Module):
    
    def __init__(self, block_size):
        super().__init__()
        self.block_size = block_size

    def forward(self, x):
        return x.view(x.size(0), -1, self.block_size, self.block_size)
        
class Flatten(nn.Module):
    
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return x.view(x.size(0), -1)

class IBGAN(nn.Module):
    
    
    def __init__(self, ngf, ndf, z_dim, r_dim, lr_G, lr_D, lr_Q, lr_E, nc=3, code_dim=0):
        
        super().__init__()
        
        self.r_dim = r_dim
        
        # Block programing with respect of paper
        
        self.Block_E = nn.Sequential(nn.Linear(z_dim, ngf*2), nn.BatchNorm1d(ngf*2), nn.ReLU(),
                                     nn.Linear(ngf*2, ngf), nn.BatchNorm1d(ngf), nn.ReLU(),
                                     nn.Linear(ngf, r_dim*2))
        
        self.Block_G = nn.Sequential(nn.Linear(r_dim+code_dim, ngf*16), nn.BatchNorm1d(ngf*16), nn.ReLU(),
                                     nn.Linear(ngf*16, 64*4*ngf), nn.BatchNorm1d(64*4*ngf), nn.ReLU(),
                                     UnFlatten(8),
                                     nn.Conv2d(ngf*4, ngf*4, 3), nn.BatchNorm2d(ngf*4), nn.ReLU(),
                                     nn.Conv2d(ngf*4, ngf*4, 3), nn.BatchNorm2d(ngf*4), nn.ReLU(),
                                     nn.ConvTranspose2d(ngf*4, ngf*4, 4, 2, 1), nn.BatchNorm2d(ngf*4), nn.ReLU(),
                                     nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1), nn.BatchNorm2d(ngf*2), nn.ReLU(),
                                     nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1), nn.BatchNorm2d(ngf), nn.ReLU(),
                                     nn.ConvTranspose2d(ngf, nc, 4, 2, 1), nn.Tanh())
        
        self.SubBlock_QD = nn.Sequential(nn.Conv2d(nc, ndf, 4, 2, 1), nn.ReLU(), # 31
                                         nn.Conv2d(ndf, ndf*2, 4, 2, 1), nn.BatchNorm2d(ndf*2), nn.ReLU(), # 15
                                         nn.Conv2d(ndf*2, ndf*4, 4, 2, 1), nn.BatchNorm2d(ndf*4), nn.ReLU(), # 7
                                         #nn.Conv2d(ndf*4, ndf*4, 3, 1, 1), nn.BatchNorm2d(ndf*4), nn.ReLU(), # 5
                                         #nn.Conv2d(ndf*4, ndf*4, 3, 1, 1), nn.BatchNorm2d(ndf*4), nn.ReLU(), # 3
                                         spectral_norm(nn.Conv2d(ndf*4, ndf*16, 4, 1, 1)), nn.BatchNorm2d(16*ndf), nn.ReLU()) # 4
       
        self.Block_Q = nn.Sequential(Flatten(),
                                     #nn.Linear(ndf*16*8, ndf*16), nn.BatchNorm1d(ndf*16), nn.ReLU(),
                                     #nn.Linear(ndf*16*16,z_dim))
                                     nn.Linear(50176, z_dim))
                                     #nn.Linear(3211264, z_dim))
        
        self.Block_D = spectral_norm(nn.Conv2d(ndf*16, 1, 2, 1, 0))
        
        self.OptD = optim.RMSprop(chain(self.Block_D.parameters(), self.SubBlock_QD.parameters()), 
                                  lr=lr_D, momentum=0.9)
        self.OptG = optim.RMSprop(self.Block_G.parameters(), lr=lr_G, momentum=0.9)
        self.OptE = optim.RMSprop(self.Block_E.parameters(), lr=lr_E, momentum=0.9)
        self.OptQ = optim.RMSprop(chain(self.Block_Q.parameters(), self.SubBlock_QD.parameters()), 
                                  lr=lr_Q, momentum=0.9)
        
    def r_sampler(self, x):
        r = self.Block_E(x)
        mu = r[:, :self.r_dim]
        var = F.softplus(r[:, self.r_dim:]) + 1e-5
        scale_tri = torch.diag_embed(var)
        return MultivariateNormal(loc=mu, scale_tril=scale_tri)
    
    def generate(self, x):
        g = self.Block_G(x)
        return g
        
    def discriminate(self, x):
        x = self.SubBlock_QD(x)
        d = self.Block_D(x)
        return d.view(-1)
        
    def forward(self, x):
        
        x = self.SubBlock_QD(x)
        Q = self.Block_Q(x)
        D = self.Block_D(x)
        return D.view(-1), Q

In [29]:
from torch.distributions.kl import kl_divergence
import torchvision.datasets as Dataset

import tqdm
from itertools import chain
from os.path import join as pjoin
import numpy as np

from torchvision import transforms
import torch.utils.data as data
import torchvision.utils as vutils

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Subset
def InfiniteSampler(n):
    # i = 0
    i = n - 1
    order = np.random.permutation(n)
    while True:
        yield order[i]
        i += 1
        if i >= n:
            np.random.seed()
            order = np.random.permutation(n)
            i = 0


class InfiniteSamplerWrapper(data.sampler.Sampler):
    def __init__(self, data_source):
        self.num_samples = len(data_source)

    def __iter__(self):
        return iter(InfiniteSampler(self.num_samples))

    def __len__(self):
        return 2 ** 31


def trans_maker(size=256):
	trans = transforms.Compose([transforms.Resize((size)),
					transforms.CenterCrop((size)), 
					transforms.ToTensor(),
          transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5)),])
	return trans

def KL_Loss(z):
	mu = z.mean()
	logvar = z.var().log()
	return -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())




In [30]:
import zipfile
with zipfile.ZipFile('drive/My Drive/img_align_celeba.zip', 'r') as zip_ref:
    zip_ref.extractall('./celeba_drive')

In [31]:
path_to_dataset = './drive/My Drive/dataset'
celeba_id = '0B7EVK8r0v71pZjFTYXZWM3FlRnM'

In [32]:
import requests

def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

destination = '../celeba'
download_file_from_google_drive(celeba_id, destination)

In [47]:
def noise_sample(r_sample=None, n_dis_c=0, dis_c_dim=0, n_z=0, batch_size=0, device=None):
    """
    Sample random noise vector for training w/ code if its needed.
    INPUT
    --------
    r_sample : Sample of intermediate representation r for cat w/ code c.
    n_dis_c : Number of discrete latent code.
    dis_c_dim : Dimension of discrete latent code.
    n_con_c : Number of continuous latent code.
    n_z : Dimension of iicompressible noise.
    batch_size : Batch Size
    device : GPU/CPU
    """

    idx = np.zeros((n_dis_c, batch_size))
    if(n_dis_c != 0):
        dis_c = torch.zeros(batch_size, n_dis_c, dis_c_dim, device=device)
        
        for i in range(n_dis_c):
            idx[i] = np.random.randint(dis_c_dim, size=batch_size)
            dis_c[torch.arange(0, batch_size), i, idx[i]] = 1.0

        dis_c = dis_c.view(batch_size, -1)

    if not r_sample==None:
      noise = torch.cat((r_sample, dis_c), dim=1)
    else:
      noise = torch.randn(batch_size, n_z, device=device)
      if(n_dis_c != 0):
        noise = torch.cat((noise, dis_c), dim=1)

    return noise, idx

In [68]:
def train(Model, args_dict):
    print('-'*25)
    print('training')
    print('-'*25)
    #dataset = Dataset.ImageFolder(root=args_dict['DATA_ROOT'], transform=trans_maker(64)) 
    #subdataset = Subset(dataset, list(range(0, args_dict['NUM_IMG'])))
    #dataloader = iter(DataLoader(subdataset, args_dict['BATCH_SIZE'], sampler=InfiniteSamplerWrapper(subdataset), num_workers=0, pin_memory=True))
    
    dataset = Dataset.CelebA(root=args_dict["DATA_ROOT"], split="all", transform=trans_maker(64), download=True)
    #dataset = torch.load(path_to_dataset)
    dataloader = DataLoader(dataset, args_dict['BATCH_SIZE'])
    
    loss_bce = nn.BCELoss()
    loss_mse = nn.MSELoss()
    
    M_r = MultivariateNormal(loc=torch.zeros(args_dict['R_DIM']).to(args_dict['device']), 
    scale_tril=torch.ones(args_dict['R_DIM'], args_dict['R_DIM']).to(args_dict['device']))
    
    D_real = D_fake = D_z_kl = G_real = Z_recon = R_kl = 0
    fixed_z = torch.randn(args_dict['BATCH_SIZE'], args_dict['Z_DIM']).to(args_dict['device'])
    
    LOG_INTERVAL = args_dict['LOG_INTERVAL']
    
    fixed_noise, _ = noise_sample(n_z=args_dict['Z_DIM'], batch_size=100, device=args_dict['device'])
    D_loss = []
    R_loss =[]
    # for n_iter in tqdm.tqdm(range(0, args_dict['ITERATIONS'])):
    
    for epoch in range(args_dict['EPOCHS']):
        Model = Model.train()
        for n_iter, (X, _) in enumerate(dataloader):
            # real_image = next(dataloader)[0].to(args_dict['device'])
            real_image = Variable(X).to(args_dict['device'])
            z, _ = noise_sample(n_z = args_dict['Z_DIM'], batch_size=args_dict['BATCH_SIZE'], device=args_dict['device'])
            r_sampler = Model.r_sampler(z)

            r_sample = r_sampler.sample()
            c_and_r, _ = noise_sample(r_sample=r_sample, n_dis_c=args_dict['NUM_DIS_C'], dis_c_dim=args_dict['DIS_C_DIM'],
                                  batch_size=args_dict['BATCH_SIZE'], device=args_dict['device'])
            
            
            g_image = Model.generate(c_and_r)
            
            Model.OptD.zero_grad()
            Model.OptQ.zero_grad()
            pred_f = Model.discriminate(g_image.detach()) # Fake prediction
            pred_r, rec_z = Model(real_image) # Real prediction and z tongo
            d_loss = (loss_bce(torch.sigmoid(pred_f), torch.ones(pred_f.size()).to(args_dict['device'])) + 
                    loss_bce(torch.sigmoid(pred_r), torch.zeros(pred_r.size()).to(args_dict['device'])))
            q_loss = KL_Loss(rec_z) # Assume a unit gaussian prior -> m(r)
            #d_loss.backward()
            total_loss = d_loss + q_loss
            total_loss.backward()
            Model.OptD.step()
            Model.OptQ.step()
    
            D_real += torch.sigmoid(pred_r).mean().item()
            D_fake += torch.sigmoid(pred_f).mean().item()
            D_z_kl += q_loss.item()
            
            Model.OptG.zero_grad()
            Model.OptE.zero_grad()
            
            pred_g, z_posterior = Model(g_image)
            
            g_loss = args_dict['LAMBDA_G']* loss_bce(torch.sigmoid(pred_g), torch.ones(pred_g.size()).to(args_dict['device']))
            # reconstruction loss of z
            recon_loss = loss_mse(z_posterior, z) # noise -> z_input
            # kl loss between e(r|z) || m(r) as a variational inference
            #kl_loss = BETA_KL * torch.distributions.kl.kl_divergence(r_likelihood, M_r).mean()
            #kl_loss = args_dict['BETA_KL']*kl_divergence(r_sampler, M_r).mean()
            kl_loss = args_dict['BETA_KL']*KL_Loss(r_sample)
            total_loss = g_loss + recon_loss + kl_loss
            total_loss.backward()
            Model.OptE.step()
            Model.OptG.step()
    
            # record the loss values
            G_real += torch.sigmoid(pred_g).mean().item()
            Z_recon += recon_loss.item()
            R_kl += kl_loss.item()
            


        Model = Model.eval()
        fixed_r_sample = Model.r_sampler(fixed_noise)
        fixed_c_and_r, _ = noise_sample(r_sample=fixed_r_sample.sample(), n_dis_c=args_dict['NUM_DIS_C'], dis_c_dim=args_dict['DIS_C_DIM'],
                                  batch_size=100, device=args_dict['device'])
        gen_data = Model.generate(fixed_c_and_r).detach().cpu()
        plt.figure(figsize=(10, 10))
        plt.axis("off")
        plt.imshow(np.transpose(vutils.make_grid(gen_data, nrow=10, padding=2, normalize=True), (1,2,0)))
        plt.show()
        
        #print("D(x): %.5f    D(G(z)): %.5f    D_kl: %.5f    G(z): %.5f    Z_rec: %.5f    R_kl: %.5f"% (D_real/LOG_INTERVAL, D_fake/LOG_INTERVAL, D_z_kl/LOG_INTERVAL, G_real/LOG_INTERVAL, Z_recon/LOG_INTERVAL, R_kl/LOG_INTERVAL))
        D_l = D_z_kl/args_dict['BATCH_SIZE']
        R_l = R_kl/args_dict['BATCH_SIZE']
        D_loss.append(D_l)
        R_loss.append(R_l)
        print("D_l: {} | R_l: {} | Epoch: {}".format(D_l, R_l, epoch))
        D_real = D_fake = D_z_kl = G_real = Z_recon = R_kl = 0
            
    return D_loss, R_loss

In [1]:
import random
seed = 1100
random.seed(seed)
torch.manual_seed(seed)

BATCH_SIZE = 64
Z_DIM = 500
R_DIM = 15
NDF = 64
NGF = 64
EPOCHS = 100
LOG_INTERVAL = 1
NUM_IMG = 1500
LAMBDA_G = 1
BETA_KL = 0.3
DATA_ROOT = "./celeba_drive/"
device = "cuda:0" if torch.cuda.is_available else "cpu"

num_dis_c = 10
dis_c_dim = 10

args_dict = {"BATCH_SIZE": BATCH_SIZE, "Z_DIM": Z_DIM, "R_DIM": R_DIM, "NDF": NDF, "NGF": NGF, "EPOCHS": EPOCHS, 
          "LAMBDA_G": LAMBDA_G, "BETA_KL": BETA_KL, "device": device, "DATA_ROOT": DATA_ROOT, "LOG_INTERVAL": LOG_INTERVAL,
          "NUM_IMG": NUM_IMG, "NUM_DIS_C": num_dis_c, "DIS_C_DIM": dis_c_dim}
    
lr = 1e-5
lr_D = 1e-7
IBGAN_model = IBGAN(NGF, NDF, Z_DIM, R_DIM, lr, lr_D, lr, lr, code_dim=num_dis_c*dis_c_dim)
IBGAN_model = IBGAN_model.to(device)

train(IBGAN_model, args_dict)

NameError: ignored

In [ ]:
!nvidia-smi